In [8]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [9]:
filename = "ABox.ttl"

In [10]:
text1 = '''CQ_1.1
What information should be returned about the heritage asset, including its name, alternative name, description, location (along with coordinates), and the temporal horizon specifying the creation event date?
'''

query1 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/01/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/01/data/>

SELECT ?asset_name ?alt_name ?description ?place_name ?coordinates ?date
WHERE {
    ?heritage_asset a tbox:HeritageAsset ;
                    tbox:hasName ?asset_name ;
                    tbox:hasAlternativeName ?alt_name ;
                    tbox:hasDescription ?description ;
                    tbox:isLocatedIn ?place .
    ?place a tbox:Place ;
            tbox:hasName ?place_name ;
            tbox:hasCoordinates ?coordinates .
    ?creation_event a tbox:CreationEvent ;
                    tbox:creates ?heritage_asset ;
                    tbox:hasTimeSpan ?time_span .
    ?time_span a tbox:TimeSpan ;
                tbox:fallsWithin ?date .
}
'''

In [11]:
text2 = '''CQ_1.2
What is the contextual information of the heritage asset in terms of its type and description?
'''

query2 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/01/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/01/data/>

SELECT ?heritage_asset ?context_type ?context_description
WHERE {
    ?heritage_asset a tbox:HeritageAsset .
    ?riskAssessment a tbox:ContextDescription ;
                    tbox:describes ?heritage_asset ;
                    tbox:hasObservation ?context .
    ?context a tbox:Observation ;
                tbox:hasParameter  ?context_type ;
                tbox:hasDescription ?context_description .
}
'''

In [12]:
text3 = '''CQ_1.3
Which documents provide information about the contextual details of the heritage asset?
'''

query3 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/01/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/01/data/>

SELECT ?heritage_asset ?context_type ?document ?document_link
WHERE {
    ?heritage_asset a tbox:HeritageAsset .
    ?riskAssessment a tbox:ContextDescription ;
                    tbox:describes ?heritage_asset ;
                    tbox:hasObservation ?context .
    ?context a tbox:Observation ;
                tbox:hasParameter  ?context_type .
    ?document a tbox:Document ;
                tbox:documents ?context ;
                tbox:hasLink ?document_link .
}
'''

In [13]:
text4 = '''CQ_1.4
Who are the stakeholders involved in the risk assessment activity related to the heritage asset, and what are their names?
'''

query4 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/01/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/01/data/>

SELECT ?heritage_asset ?stakeholder_name
WHERE {
    ?heritage_asset a tbox:HeritageAsset .
    ?risk_assessment_activity a tbox:ContextDescription ;
                                tbox:describes ?heritage_asset ;
                                tbox:hasParticipant ?stakeholder .
    ?stakeholder a tbox:Stakeholder ;
                tbox:hasName ?stakeholder_name .
}
'''

In [14]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text4, query4)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_1.1
Return the information about the heritage asset, including its name, alternative name, its description, the place it is located in (as well as its coordinates), and the time-span temporal horizon that identifies the creation event it has been dated to.

+-------------------------+-------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------+--------------------------------------+-------------------------+
| asset_name         